## Get zip files of building records using cesmd fetcher

In [1]:
import search
from pathlib import Path

REQUEST URL: https://www.strongmotioncenter.org/wserv/records/query?email=cchern%40berkeley.edu&rettype=dataset&download=P&groupby=station&abandoned=false&eventid=ci3144585&orderby=epidist-asc&nodata=404


In [2]:

print(search.__file__)


/Users/guonaiqi/Documents/GitHub/cesmd-fetcher/cesmd/search.py


In [3]:
from buildings import BUILDINGS

building_dir = Path("CSMIP/buildings")
if not building_dir.exists():
    Path.mkdir(building_dir)

In [6]:
update_files = True

In [7]:
if update_files:
    for cesmd in BUILDINGS.keys():
        print(BUILDINGS[cesmd])
        search.get_records(str(building_dir)+"/"+cesmd, "cchern@berkeley.edu", station_code=cesmd, process_level="processed", include_inactive=True)

{'cesmd': 'CE24602', 'name': 'Los Angeles - 52-story Office Bldg', 'predictors': [{'name': 'SRIM_H1', 'description': '1st horizontal configuration.', 'metrics': ['SPECTRAL_SHIFT_IDENTIFICATION'], 'entry_point': ['/Users/guonaiqi/miniforge3/envs/xara/bin/python', '-m', 'mdof', 'srim'], 'config': {'decimate': 1, 'order': 12, 'horizon': 100, 'inputs': [2, 5], 'outputs': [8, 10, 13, 16, 19]}}, {'name': 'SRIM_H2', 'description': '2nd horizontal configuration.', 'metrics': ['SPECTRAL_SHIFT_IDENTIFICATION'], 'entry_point': ['/Users/guonaiqi/miniforge3/envs/xara/bin/python', '-m', 'mdof', 'srim'], 'config': {'decimate': 1, 'order': 12, 'horizon': 100, 'inputs': [3, 6], 'outputs': [9, 11, 14, 17, 20]}}, {'name': 'FDD_H1', 'description': '1st horizontal configuration.', 'metrics': ['SPECTRAL_SHIFT_IDENTIFICATION'], 'entry_point': ['/Users/guonaiqi/miniforge3/envs/xara/bin/python', '-m', 'mdof', 'fdd'], 'config': {'period_band': [0.1, 8], 'outputs': [8, 10, 13, 16, 19]}}, {'name': 'FDD_H2', 'desc

In [4]:
search.get_records(
    output="tmp_northridge",
    email="cchern@berkeley.edu",
    return_type="dataset",
    process_level="processed",
    group_by="station",
    eventid="ci3144585",
    unpack=True,
)


REQUEST URL: https://www.strongmotioncenter.org/wserv/records/query?email=cchern%40berkeley.edu&rettype=dataset&download=P&groupby=station&abandoned=false&eventid=ci3144585&orderby=epidist-asc&nodata=404


('/Users/guonaiqi/Documents/GitHub/cesmd-fetcher/cesmd/tmp_northridge',
 ['Northridge_17Jan1994_LA00073P.ZIP',
  'northridge_17jan1994_ce22t04p.zip',
  'Northridge_17Jan1994_LA00087P.ZIP',
  'Northridge_17Jan1994_LA00074P.ZIP',
  'Northridge_17Jan1994_LA00089P.ZIP',
  'northridge_17jan1994_ce25282p.zip',
  'Northridge_17Jan1994_LA00045P.ZIP',
  'Northridge_17Jan1994_LA00011P.ZIP',
  'northridge_17jan1994_ce24303p.zip',
  'northridge_17jan1994_ce23542p.zip',
  'Northridge_17Jan1994_LA00016P.ZIP',
  'Northridge_17Jan1994_LA00020P.ZIP',
  'northridge_17jan1994_wr00584p.zip',
  'northridge_17jan1994_ce23574p.zip',
  'Northridge_17Jan1994_LA00018P.ZIP',
  'Northridge_17Jan1994_LA00044P.ZIP',
  'Northridge_17Jan1994_LA00088P.ZIP',
  'Northridge_17Jan1994_LA00075P.ZIP',
  'Northridge_17Jan1994_LA00081P.ZIP',
  'northridge_17jan1994_ce11625p.zip',
  'northridge_17jan1994_ce14368p.zip',
  'Northridge_17Jan1994_LA00086P.ZIP',
  'Northridge_17Jan1994_LA00072P.ZIP',
  'northridge_17jan1994_ce13589

## Move records to top levels of unzipped folders

In [6]:
import glob
from zipfile import ZipFile

In [7]:
extract_new = True

In [8]:
if extract_new:
    for cesmd_zip in glob.glob(str(building_dir)+"/*.zip"):
        newdir = cesmd_zip[:-4]
        with ZipFile(cesmd_zip) as zObject: 
            zObject.extractall(path="./"+newdir)

In [9]:
move_new = True

In [10]:
if move_new:
    import os

    def delete_empty_folders(root):
        deleted = set()
        for current_dir, subdirs, files in os.walk(root, topdown=False):
            still_has_subdirs = False
            for subdir in subdirs:
                if os.path.join(current_dir, subdir) not in deleted:
                    still_has_subdirs = True
                    break
            if not any(files) and not still_has_subdirs:
                os.rmdir(current_dir)
                deleted.add(current_dir)
        return deleted

    for cesmd in BUILDINGS.keys():
        for record in glob.glob(str(building_dir)+"/"+cesmd+"/*/*/*.zip"):
            os.replace(record, str(building_dir)+"/"+cesmd+"/"+os.path.basename(record))
    
    delete_empty_folders(".")
    

In [11]:
summarize_new = True

In [12]:
if summarize_new:
    import quakeio
    import json
    for cesmd in BUILDINGS.keys():  
        events = []
        for file in glob.glob(str(building_dir)+"/"+cesmd+"/*.zip"):
            print(f"Reading {file}")
            try:
                events.append(
                    quakeio.read(file, summarize=True).serialize(serialize_data=False)
                )
            except:
                print(f"failed to read {file}")
        with open(str(building_dir)+"/"+cesmd+"_meta.json", "w") as writefile:
            json.dump(events, writefile)

In [12]:
building_names = [{'cesmd': cesmd, 'name': name} for cesmd,name in BUILDINGS.items()]

In [13]:
with open(str(building_dir)+"/building_names.json", "w") as writefile:
    json.dump(building_names, writefile)

In [14]:
# Don't worry about this; it's for making webpages of plots of building accelerations

for cesmd in BUILDINGS.keys():  
    print(f"""
    <div id='{cesmd}_PeakAccelHistDiv' style="width:800px;height:400px;"></div>
    <div id='{cesmd}_PeakAccelPlotDiv' style="width:800px;height:400px;"></div>""", end="")


    <div id='CE24602_PeakAccelHistDiv' style="width:800px;height:400px;"></div>
    <div id='CE24602_PeakAccelPlotDiv' style="width:800px;height:400px;"></div>
    <div id='CE24386_PeakAccelHistDiv' style="width:800px;height:400px;"></div>
    <div id='CE24386_PeakAccelPlotDiv' style="width:800px;height:400px;"></div>
    <div id='CE58483_PeakAccelHistDiv' style="width:800px;height:400px;"></div>
    <div id='CE58483_PeakAccelPlotDiv' style="width:800px;height:400px;"></div>
    <div id='CE24579_PeakAccelHistDiv' style="width:800px;height:400px;"></div>
    <div id='CE24579_PeakAccelPlotDiv' style="width:800px;height:400px;"></div>
    <div id='CE24322_PeakAccelHistDiv' style="width:800px;height:400px;"></div>
    <div id='CE24322_PeakAccelPlotDiv' style="width:800px;height:400px;"></div>
    <div id='CE23634_PeakAccelHistDiv' style="width:800px;height:400px;"></div>
    <div id='CE23634_PeakAccelPlotDiv' style="width:800px;height:400px;"></div>
    <div id='CE89494_PeakAccelHistDiv' 